In [1]:
import pandas as pd
import tensorflow as tf

2025-01-23 10:23:28.628721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9373] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-23 10:23:28.628795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-23 10:23:28.630003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1534] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-23 10:23:28.637010: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization,GlobalAveragePooling2D

In [3]:
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [4]:
#listing all the classes
import os
labels=os.listdir('garbage_classification')
print(labels)

['brown-glass', 'shoes', 'trash', 'battery', 'white-glass', 'clothes', 'metal', 'cardboard', 'paper', 'green-glass', 'plastic', 'biological']


In [5]:
dataset_count = {}
for label in labels:
    label_path = os.path.join('garbage_classification', label)
    count = len(os.listdir(label_path))
    dataset_count[label] = count

print(dataset_count)

{'brown-glass': 607, 'shoes': 1977, 'trash': 697, 'battery': 945, 'white-glass': 775, 'clothes': 5325, 'metal': 769, 'cardboard': 891, 'paper': 1050, 'green-glass': 629, 'plastic': 865, 'biological': 985}


In [6]:
# making train and test directory
os.makedirs('dataset', exist_ok=True)
train_dir = os.path.join('dataset', "train")
test_dir = os.path.join('dataset', "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [7]:
#splittting the dataset into 80:20 ratio
import shutil
import random
for label in os.listdir('garbage_classification'):
    label_path = os.path.join('garbage_classification', label)
    if os.path.isdir(label_path):
        os.makedirs(os.path.join(train_dir, label), exist_ok=True) # creating directories of each class train
        os.makedirs(os.path.join(test_dir, label), exist_ok=True) # creating directories of each class in test
        images = os.listdir(label_path)
        random.shuffle(images)

        #spliting the dataset
        split_idx = int(len(images) * 0.8)
        train_images = images[:split_idx]
        test_images = images[split_idx:]

        #copying the images
        for img in train_images:
            shutil.copy(os.path.join(label_path, img), os.path.join(train_dir, label, img))
        for img in test_images:
            shutil.copy(os.path.join(label_path, img), os.path.join(test_dir, label, img))


In [8]:
# setting train and test path
train_path = os.path.join('dataset', "train")
test_path = os.path.join('dataset', "test")
print(train_path)
print(test_path)

dataset/train
dataset/test


In [9]:
#Setting the image height and width
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

In [10]:
#resizing the images in train directory
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

Found 12409 files belonging to 12 classes.


2025-01-23 10:27:36.090599: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1926] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8048 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:bd:00.0, compute capability: 8.0


In [11]:
#resizing the images in test directory
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

Found 3106 files belonging to 12 classes.


In [12]:
class_names = train_dataset.class_names
class_names

['battery',
 'biological',
 'brown-glass',
 'cardboard',
 'clothes',
 'green-glass',
 'metal',
 'paper',
 'plastic',
 'shoes',
 'trash',
 'white-glass']

In [13]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [14]:
#preprocessing the train and test dataset
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

In [15]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
from tensorflow.keras.layers import BatchNormalization, Dropout

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
from tensorflow.keras.applications import ResNet50

In [19]:
from tensorflow.keras.models import Model

In [20]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 6s 0us/step


In [21]:
# Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [22]:
# Add a fully connected layer with 256 units and ReLU activation
x = Dense(256, activation='relu')(x)

In [23]:
predictions = Dense(12, activation='softmax')(x)

In [24]:
# Construct the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [25]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [26]:
# Compile the model
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [28]:
model.build(input_shape=(BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, 3))

In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [30]:
os.makedirs('modes')

In [31]:
# Checkpoint highest accuracy
#will save the best version of the model with highest accuracy
checkpoint_high_acc = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"{'modes'}/base_model_highest_accuracy.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

# Checkpoint lowest loss
#will save the best version of the model with lowest loss
checkpoint_low_loss = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"{'modes'}/base_model_lowest_loss.keras",
    monitor="val_loss",
    save_best_only=True,
    mode="min",
    verbose=1
)

In [32]:
import time
start_time = time.time()
history = model.fit(train_dataset, 
                    epochs=50, 
                    validation_data=test_dataset, 
                    callbacks=[checkpoint_high_acc, checkpoint_low_loss, early_stopping])
end_time = time.time()
print("Training is stopped")

Epoch 1/50


2025-01-23 10:35:30.525802: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:467] Loaded cuDNN version 90000
2025-01-23 10:35:33.534108: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc8552b93c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-01-23 10:35:33.534160: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 2g.10gb, Compute Capability 8.0
2025-01-23 10:35:33.539782: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737628533.646833    1019 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


388/388 [==============================] - ETA: 0s - loss: 0.8274 - accuracy: 0.7482
Epoch 1: val_accuracy improved from -inf to 0.03509, saving model to modes/base_model_highest_accuracy.keras

Epoch 1: val_loss improved from inf to 4.40585, saving model to modes/base_model_lowest_loss.keras
388/388 [==============================] - 125s 208ms/step - loss: 0.8274 - accuracy: 0.7482 - val_loss: 4.4058 - val_accuracy: 0.0351
Epoch 2/50
388/388 [==============================] - ETA: 0s - loss: 0.1588 - accuracy: 0.9469
Epoch 6: val_accuracy improved from 0.69350 to 0.82936, saving model to modes/base_model_highest_accuracy.keras

Epoch 6: val_loss improved from 1.31547 to 0.58556, saving model to modes/base_model_lowest_loss.keras
388/388 [==============================] - 68s 175ms/step - loss: 0.1588 - accuracy: 0.9469 - val_loss: 0.5856 - val_accuracy: 0.8294
Epoch 7/50
388/388 [==============================] - ETA: 0s - loss: 0.1345 - accuracy: 0.9559
Epoch 7: val_accuracy did not

In [ ]:
print('Computational Time =', (end_time - start_time)/60, 'minutes')

In [33]:
model.save('resnet50.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
model.save('resnet50.keras')